# Tutorial on how to prepare a protocol file for `VaspMultiStageWorkChain`

## S protocol:
This protocol has one stage:
1. S: final static calculation for having accurate energies!

## Step 1: Loading modules:
We need to load following modules:
* `os`: to help writing the results file to desired location
* `yaml`: to convert python dictionary to yaml file
* `Structure` class from `pymatgen`: to generate `pymatgen` `Structure` object
* All classes from `VASP` sets of `pymatgen`: to get pre-defined and tested sets as starting point.

In [1]:
import os
import yaml
from pymatgen import Structure
from pymatgen.io.vasp.sets import *

## Step 2: Generate `Structure` object
The inputs sets from `pymatgen` require a `Structure` object to be passed through to work.
It can be any structure~

In [2]:
structure = Structure.from_file('../examples/files/Li_bcc_222.vasp')

## Step 3: Get the `INCAR` from sets and polish them
`pymatgen` comes with a handful of input sets. Below, we use ones used `Materials Project` for `static` and `relax` calculations. We then remove `'@module', '@class', 'MAGMOM'` keys that comes with them. 

**Notes**:
* We remove `MAGMOM` to remove dependency of protocol to the structure. It will be dealt with within the workchain.

In [3]:
incar_stage_0 = MPStaticSet(structure).incar.as_dict()

for dicts in [incar_stage_0]:
    for keys in ['@module', '@class', 'MAGMOM']:
        del dicts[keys]

/home/pezhman/.local/software/anaconda3/envs/aiidadev/lib/python3.6/site-packages/pymatgen/io/vasp/sets.py:592: BadInputSetWarning: Relaxation of likely metal with ISMEAR < 1 detected. Please see VASP recommendations on ISMEAR for metals.
  BadInputSetWarning,


## Step 4: Generating a dictionary with all stages
Herein, we define a new `object` and put the `INCAR`s under `stage_x` keys. Note that put them in the order you would like them to be called within the workchain.

In [4]:
all_stages = {}
all_stages['stage_0'] = incar_stage_0

## Step 5 (optional): Modifying stages
In case you would like to modify some parameters from `INCAR` to differ from the provided values from sets, here you may do it before dumping them to the `yaml` file.

### Modifications for `final_static` run

In [5]:
# We change it to Fast as initial setting
all_stages['stage_0']['ALGO'] = 'Fast'
# Even tighter compared to relax
all_stages['stage_0']['EDIFF'] = 1e-7
# Let's have it at 700
all_stages['stage_0']['ENCUT'] = 700
# Change to Gaussian smearing as initial setting
all_stages['stage_0']['ISMEAR'] = 0
# In the final static, we may need these files for possible Bader/DDEC charge analysis!
all_stages['stage_0']['LAECHG'] = True
all_stages['stage_0']['LCHARG'] = True
all_stages['stage_0']['LVHAR'] = True
# Increase number of electronic steps!
all_stages['stage_0']['NELM'] = 200

## Step 6: Checking the final results

In [6]:
print(yaml.dump(all_stages))

stage_0:
  ALGO: Fast
  EDIFF: 1.0e-07
  ENCUT: 700
  IBRION: -1
  ICHARG: 0
  ISIF: 3
  ISMEAR: 0
  ISPIN: 2
  LAECHG: true
  LCHARG: true
  LORBIT: 11
  LREAL: Auto
  LVHAR: true
  LWAVE: false
  NELM: 200
  NSW: 0
  PREC: Accurate
  SIGMA: 0.05



## Step 7: Writing the results to a `yaml` file
As we would like to use them with our workchain, we can write it to protocol folder.

In [7]:
# We need to give a name to file.
# My suggestion is:
# S for static, R for relax, N for non-scf static, m for md
# These can be follow by 0 for initial
# In the case of relax: R can be followed by ISIF too.
# S0R3S: initial static --> relax(isif 3) --> final static
# S0R3SN: static --> relax(isif 3) --> static --> NSCF static
# R02R3S: initial relax(isif 2) --> relax(isif 3) --> static

# So here we would have: S0R3S
protocol_name = 'S.yaml'
this_directory = os.getcwd()
yaml_path = os.path.join(this_directory, '..', 'aiida_bjm', 'workchains', 'protocols', 'vasp', protocol_name)
with open(os.path.join(yaml_path), 'w') as file:
    yaml.dump(all_stages, file)